In [33]:
import pandas as pd
from rapidfuzz import process, fuzz

In [7]:
# Specify the path to your Excel file
file_path = '../data/Cruises_Original_20241121.xlsx'

In [15]:
df = pd.read_excel(file_path, header=3)  # Starts reading from the 3rd row

In [16]:
df.head()

,Voyage #,Year,Trip,Ship,Dates,Unnamed: 5,Stops / Ports,Unnamed: 7,comments,Unnamed: 9,CATALOG #,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1932-1,FIRST,Cal. State,1931-12-31 00:00:00,NaN,California City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,15-17 Jan 32,NaN,"Balboa, Panama",NaN,NaN,NaN,04 . 01 .,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,17-18 Jan 32,NaN,"St. Elmo Bay, Panama",NaN,anchored at Las Perlas Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Annual #,1,NaN,24-27 Jan 32,NaN,"Callao, Peru",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# Drop completely empty columns

df_cleaned = df.dropna(how='all', axis=1)
df_cleaned = df_cleaned.dropna(how='all', axis=0)



In [49]:
# Remove rows containing "cancelled" (case-insensitive) in any column

cancelled_filter = df_cleaned.apply(lambda row: row.astype(str).str.contains("cancelled", case=False).any(), axis=1)

df_filtered = df_cleaned[~cancelled_filter]


In [45]:
# Display the cleaned and filtered data

df_filtered.head()

,Voyage #,Year,Trip,Ship,Dates,Unnamed: 5,Stops / Ports,Unnamed: 7,comments,CATALOG #,Unnamed: 11,Unnamed: 12,Unnamed: 14,Unnamed: 18,Standardized Stops/Ports
1,1.0,1932-1,FIRST,Cal. State,1931-12-31 00:00:00,NaN,California City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,15-17 Jan 32,NaN,"Balboa, Panama",NaN,NaN,04 . 01 .,1.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,17-18 Jan 32,NaN,"St. Elmo Bay, Panama",NaN,anchored at Las Perlas Islands,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Annual #,1,NaN,24-27 Jan 32,NaN,"Callao, Peru",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,02-08 Feb 32,NaN,"Valparaiso, Chile",NaN,and Santiago (by train),NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Load the reference table (single column of standard port names)
reference_path = '../data/portsSpreadsheet.csv'
reference_df = pd.read_csv(reference_path)

In [68]:
# Create combined reference strings for matching
reference_df['Combined Port'] = reference_df['Cleaned Port'] + ', ' + reference_df['Country']
reference_ports = reference_df['Combined Port'].tolist()
reference_port_cities = reference_df['Cleaned Port'].tolist()


In [69]:
def get_best_match(port):
    # Extract the best match from the list
    result = process.extractOne(port, reference_ports, scorer=fuzz.ratio)
    
    # Check if a result is found
    if result:
        match, score = result[0], result[1]  # Extract match and score
        if score > 80:
            return match
        else:
            # Extract the best match from the list
            result = process.extractOne(port, reference_port_cities, scorer=fuzz.ratio)
            
            # Check if a result is found
            if result:
                match, score = result[0], result[1]  # Extract match and score
                return match if score > 80 else 'Unknown'  # Use a threshold of 80
    else:
            # Extract the best match from the list
            result = process.extractOne(port, reference_port_cities, scorer=fuzz.ratio)
            
            # Check if a result is found
            if result:
                match, score = result[0], result[1]  # Extract match and score
                return match if score > 80 else 'Unknown'  # Use a threshold of 80
    return 'Unknown'  # Return 'Unknown' if no match is found


In [70]:
main_df = df_filtered

In [71]:
# Apply fuzzy matching
main_df.loc[:, ['Standardized Port']] = main_df['Stops / Ports'].apply(get_best_match)


In [72]:
main_df['Standardized Port'].value_counts()

Standardized Port
Unknown                     364
Vallejo                     189
San Francisco, USA           38
San Diego                    38
Balboa, Panama               28
                           ... 
Boston, USA                   1
Honiara, Soloman Islands      1
Lisbon, Portugal              1
Port Everglades, USA          1
Galveston, USA                1
Name: count, Length: 153, dtype: int64

In [73]:
# Extract unique pairs of original and standardized stops/ports
unique_ports = main_df[['Stops / Ports', 'Standardized Port']].drop_duplicates()

# Filter rows where 'Standardized Stops/Ports' is 'Unknown'
unknown_ports = main_df[main_df['Standardized Port'] == 'Unknown'][['Stops / Ports', 'Standardized Port']].drop_duplicates()

# Combine both datasets (unique pairs and unknown ports)
all_ports = pd.concat([unique_ports, unknown_ports]).drop_duplicates()

# Split 'Standardized Port' into 'City' and 'Country'
all_ports[['City', 'Country']] = all_ports['Standardized Port'].str.split(', ', expand=True)

# Save to a CSV for review
all_ports.to_csv('all_ports_mapping.csv', index=False)

# Display the result
print(all_ports)

              Stops / Ports     Standardized Port             City   Country
1           California City  California City, USA  California City       USA
2            Balboa, Panama        Balboa, Panama           Balboa    Panama
3      St. Elmo Bay, Panama               Unknown          Unknown      None
4              Callao, Peru          Callao, Peru           Callao      Peru
5         Valparaiso, Chile     Valparaiso, Chile       Valparaiso     Chile
...                     ...                   ...              ...       ...
1559            Lahaina, HI               Unknown          Unknown      None
1567       Lisbon, Portugal      Lisbon, Portugal           Lisbon  Portugal
1568  Ponta Delgada, Azores               Unknown          Unknown      None
1569     Port Everglade, FL  Port Everglades, USA  Port Everglades       USA
1570          Galveston, TX        Galveston, USA        Galveston       USA

[341 rows x 4 columns]
